In [2]:
import pandas as pd

# Data Cleaning

- Replacing Nan values
- Using mean for numeric 
- Using mode for textual data

In [ ]:
bios_df=pd.read_csv('./uncleaned_datasets/bios.csv')
results_df=pd.read_csv('./uncleaned_datasets/results.csv')

### Replace height and weight null values with their respective mean

In [5]:
# Get means rounded to 2 decimal places
height_mean = round(bios_df['height_cm'].mean(), 1)
weight_mean = round(bios_df['weight_kg'].mean(), 1)

# Replace NaN values with rounded means
bios_df['height_cm'] = bios_df['height_cm'].fillna(height_mean)
bios_df['weight_kg'] = bios_df['weight_kg'].fillna(weight_mean)

# Verify the changes and see the rounded means
print("Average height:", height_mean)  # Should show 176.33
print("Average weight:", weight_mean)  # Should show 71.89

Average height: 176.3
Average weight: 71.9


### Handling missing dates

In [8]:
from datetime import datetime, timedelta

def clean_dataset(df):
    """
    Cleans the dataset by handling missing values in born_date and died_date columns.
    
    Parameters:
    df (pandas.DataFrame): The input dataset
    
    Returns:
    pandas.DataFrame: The cleaned dataset
    """
    # Handle missing born_date
    df['born_date'] = pd.to_datetime(df['born_date'], errors='coerce')
    df['died_date'] = pd.to_datetime(df['died_date'], errors='coerce')
    
    # Fill in missing dates
    df.loc[df['born_date'].isna() & df['died_date'].notna(), 'born_date'] = df.loc[df['born_date'].isna() & df['died_date'].notna(), 'died_date'] - timedelta(days=60*365 + 2*30 + 3)
    df.loc[df['died_date'].isna() & df['born_date'].notna(), 'died_date'] = df.loc[df['died_date'].isna() & df['born_date'].notna(), 'born_date'] + timedelta(days=60*365 + 2*30 + 3)
    df.loc[df['born_date'].isna() & df['died_date'].isna(), ['born_date', 'died_date']] = ['1898-07-16', '1969-11-27']
    
    # Convert to datetime
    df['born_date'] = pd.to_datetime(df['born_date'])
    df['died_date'] = pd.to_datetime(df['died_date'])
    
    return df

In [9]:
bios_df=clean_dataset(bios_df)

### Handling missing region information

In [13]:
def replace_nan_with_mode(df):
    """
    Replace NaN values in the DataFrame with the mode of each column, except for 'NOC' which is replaced with 'FRA'.
    
    Parameters:
    df (pandas.DataFrame): The input DataFrame
    
    Returns:
    pandas.DataFrame: The DataFrame with NaN values replaced
    """
    # Replace NaN in 'NOC' column with 'FRA'
    df['NOC'] = df['NOC'].fillna('FRA')
    
    # Replace NaN in other columns with mode
    for col in ['born_city', 'born_region', 'born_country']:
        df[col] = df[col].fillna(df[col].mode().iloc[0])
    
    return df

In [14]:
bios_df=replace_nan_with_mode(bios_df)

### Below output show, no null values remain in bios_df

In [16]:
print(bios_df.isna().sum())

athlete_id      0
name            0
born_date       0
born_city       0
born_region     0
born_country    0
NOC             0
height_cm       0
weight_kg       0
died_date       0
dtype: int64


# Handling `result_df` dataset

In [19]:
print(results_df.isna().sum())

year            2601
type            2601
discipline         1
event              0
as                 0
athlete_id         0
noc                1
team          186694
place          25215
tied               0
medal         264269
dtype: int64


In [20]:
results_df.columns

Index(['year', 'type', 'discipline', 'event', 'as', 'athlete_id', 'noc',
       'team', 'place', 'tied', 'medal'],
      dtype='object')

### Handling nan values in these columns: `['year', 'type', 'medal', 'place', 'team', 'discipline']`

In [22]:

def replace_nan_values(df):
    """
    Replace NaN values in the DataFrame with appropriate measures of central tendency.
    
    Parameters:
    df (pandas.DataFrame): The input DataFrame
    
    Returns:
    pandas.DataFrame: The DataFrame with NaN values replaced
    """
    # Replace NaN in 'year', 'type', 'medal', 'place', 'team', 'discipline' columns with mode
    for col in ['year', 'type', 'medal', 'place', 'team', 'discipline']:
        df[col] = df[col].fillna(df[col].mode().iloc[0])
    
    # Replace NaN in 'noc' column with 'FRA'
    df['noc'] = df['noc'].fillna('FRA')
    
    return df

In [23]:
results_df=replace_nan_values(results_df)

In [25]:
print(results_df.isna().sum())

year          0
type          0
discipline    0
event         0
as            0
athlete_id    0
noc           0
team          0
place         0
tied          0
medal         0
dtype: int64


### Now null count is 0 as shown above in output.

### Saving data after cleaning.

In [26]:
bios_df.to_csv('./cleaned_data/bios.csv', index=False)
results_df.to_csv('./cleaned_data/results.csv', index=False)